In [ ]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
import plotly.express as px
from selenium import webdriver


In [ ]:
driver = webdriver.Chrome()

### Fonction pour générer l'URL en fonction du terme recherché

In [ ]:
def url_by_term(search_term,num_page):
    form = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_1&page="+str(num_page)
    term = search_term.replace(" ", "+")
    return form.format(term)

term = "portable"


### Extraction des liens

In [ ]:
#Ici j'extraie les liens de redirections
links = []
nb_pages_searching_result = 60
for i in range(nb_pages_searching_result):
    
    url = url_by_term(term,i)
    driver.get(url)

    bs = BeautifulSoup(driver.page_source,'html.parser')
    results = bs.find_all('h2', {'class':'a-size-mini a-spacing-none a-color-base s-line-clamp-4'})
    
    if len(results)==0:
        results = bs.find_all('h2', {'class':'a-size-mini a-spacing-none a-color-base s-line-clamp-2'})

    for r in results:
        
        links.append('https://www.amazon.com'+r.find('a')['href'])

#Ici j'extraie les vrais lien à partir des liens de redirections
c=0
newlink = []
for x in links: 
    
    driver.get(x)
    newlink.append(driver.current_url)
    c = c+1
    print(c)
    


### Extraction d'avis et de commentaire

In [ ]:

for j in range(len(newlink)):
    newlink[j] = newlink[j].replace("dp","product-reviews")

nb_pages_review = 20
review = []    
list_links=[]
positif=[]
c1=0
for g in newlink:
    c1 = c1+1
    print(c1)
    for i in range(nb_pages_review):
        for j in ['positive','critical']:
            list_links.append(g+"&filterByStar="+j+"&pageNumber="+str(i))
            #driver.get(g+"&filterByStar="+j+"&pageNumber="+str(i))
            driver.get(g+"&filterByStar="+j+"&pageNumber="+str(i))

            bs = BeautifulSoup(driver.page_source,'html.parser')
            results = bs.find_all('span', {'data-hook':'review-body'})

           
            for r in results:
                review.append(r.find('span').text)


                if j == 'positive':
                    positif.append(1)
                if j == 'critical':
                    positif.append(0)



### Raccourcis des commentaires (Deux premières et deux dernières phrases)

In [ ]:
review_list = []
for i in review:
    
    liste= re.split('[?.!;]', i)
    
    if len(liste)>4:
        j = liste[0]+"."+liste[1]+"."+liste[-3]+"."+liste[-2]+"."
    if len(liste)==4:
        j = liste[0]+"."+liste[1]+"."+liste[2]+"."
    if len(liste)==3:
        j = liste[0]+"."+liste[1]+"."
    if len(liste)==2:
        j = liste[0]+"."
    if len(liste)==1:
        j = liste[0]+"."
        
    review_list.append(j)
    
review_list

### Nettoyage et Mise en Format JSON

In [ ]:

d = {'review' : review_list, 'positive': positif}
review_df = pd.DataFrame(data=d)
review_df = review_df.replace('\n','', regex=True)
review_df = review_df.replace('\u2019','`', regex=True)
review_df = review_df.replace('\u201d','', regex=True)
review_df = review_df.replace('\u201c','', regex=True)
review_df = review_df.replace('\ud83d','', regex=True)
review_df = review_df.replace('\udc22','', regex=True)
review_df = review_df.replace('\/',' ', regex=True)

review_df.drop_duplicates(keep = 'first', inplace=True)

review_df.to_json(r'C:\Users\cleme\OneDrive\Documents\Python Scripts\Amazon Web Scrapping\review_amazon_'+term.replace(' ','')+str(time.strftime("%d_%B_%Y"))+'.json',orient='records')